# Exploring spatial and biodiversity data

```{admonition} Summary
:class: hint
This walkthrough provides a complete data processing example from start to finish.

The example is based on two data sources:
- **Biodiversity data** from [Lebendiger Atlas der Natur Deutschlands (LAND)](https://land.gbif.de/): Species observations as coordinate-based vector data.
- **Spatial data** from [IOER Monitor](https://www.ioer-monitor.de/): Aggregated and processed land use data that is provided in raster format (WCS).

```

```{admonition} Summary
:class: hint
While Python offers multiple ways to process data, we will focus on a single approach without exploring alternative routes. Where further details are useful, cross-references to the reference documentation (Part III) or external sources are provided.
```

## Biodiversity data

[LAND - Lebendiger Atlas der Natur Deutschlands](https://land.gbif.de/) compiles species observation data in Germany. It displays selected occurrence datasets that are publicly available through the [Global Biodiversity Information Facility (GBIF)](Gbif.org). GBIF aims to provide open access to biodiversity data worldwide. This connection enables users to explore Germany’s biodiversity data via LAND while also accessing and utilizing selected datasets from GBIF’s global database.

There are two ways to find and download data:
- **Via web browser**: Search, explore and download individual data files directly from [LAND](https://land.gbif.de/) using its occurrence search, geo viewer, and download options.
- **Via code**: Use the [GBIF Application Programming Interface (API) Reference](https://techdocs.gbif.org/en/openapi/) to access data programmatically.

For both methods, and especially for larger downloads, registration is required.

```{figure} https://images.naturalis.nl/original/csr%2FD80_8153.jpg
:name: sparrow-graphic

English Sparrow, *Passer domesticus* (GBIF.org).
```

```{admonition} Using GBIF API
:class: warning
 

```

## Spatial data - IOER Monitor of Settlement and Open Space Development (IOER Monitor)

The IOER Monitor of Settlement and Open Space Development (short [IOER Monitor](https://monitor.ioer.de/)) is a research data infrastructure provided by the Leibniz Institute of Ecological Urban and Regional Development (IOER). It offers insights into land use structure, development, and landscape quality in Germany. Indicators and data can be explored and visualized in an [interactive geo viewer](https://monitor.ioer.de).
All IOER Monitor data is available through **Web Feature Service (WFS)** and **Web Coverage Service (WCS)**, allowing users to retrieve spatial data in standardized formats.

There are two ways to access and download data:

- **Via web browser**: Login, search and download individual data files directly from the IOER Monitor's download services.
- **Via code**: Use the Monitor API to access data programmatically.

In both cases, [registration and/or login](https://monitor.ioer.de/monitor_api/) are required.

```{admonition} Using the Monitor API
:class: warning

If you want to use the Monitor API, you need to register:

1. Register at [monitor.ioer.de/monitor_api/signup](https://monitor.ioer.de/monitor_api/signup).
2. Generate a personal API key in your account settings.
3. Store your personal API key in a file called `.env` as `API_KEY=xyz`
```

 ```{figure} https://www.ioer-monitor.de/fileadmin/user_upload/monitor/img/Ergebnisse/siedlungsdichte.png
:name: monitor-graphic

Wie dicht leben wir? [IOER Monitor data](https://www.ioer-monitor.de/ergebnisse/analyseergebnisse/wie-dicht-leben-wir/).
```

## Example research question

In the next section, we demonstrate how to combine spatial and biodiversity data in a practical example.

Specifically, we will investigate whether there is a positive correlation between *Passer domesticus* (common name: `House Sparrow`, `English Sparrow`) density and settlement areas. Since house sparrows are known to prefer urban environments, this pattern should be visible using data from the IOER Monitor and LAND data.

Additionally, we will examine population trends for house sparrows in Saxony. According to {cite:ts}`salek_house_2015`, an overall decline in house sparrow populations has been observed:

> _Populations of House and Tree Sparrows have rapidly declined in various breeding habitats throughout their European distribution range; however, the strongest decline was recorded within urban environments._
{cite:p}`salek_house_2015`

But is this also true for Saxony? [Let’s explore whether the data supports this trend...](202_data_retrieval)

### Example: Finding the scientific name of a species

Let’s say we only have a species' common name — such as `English Sparrow`, which is also known as `House Sparrow`, `Tree Sparrow` - and want to find its scientific name. The [GBIF.org species search](https://www.gbif.org/species/search?q=english%20sparrow) provides the correct answer: `Passer domesticus`. 

Since [GBIF.org](https://www.gbif.org) uses its own API internally, we can likely access the same data and information programmatically. Let's explore how!

```{figure} https://images.naturalis.nl/original/csr%2FD80_8153.jpg
:name: sparrow-graphic

English Sparrow, *Passer domesticus* (GBIF.org).
```

### GBIF API Reference

Retrieving data from APIs requires a specific syntax, which varies for each service. Here are two key concepts:

- **Endpoint**: APIs provide commonly URLs (endpoints) that return structured data (e.g., in **JSON** format). The base URL is for the GBIF API is [https://api.gbif.org/](https://api.gbif.org/). 
- **Authentication**: Many APIs require an authentication, but some — like GBIF — allow limited access without it. However, there are APIs that don´t need authentication (e.g. Instagram, commons.wikimedia.org).


```{dropdown} How do I know how to work with GBIF API?
Good APIs have a documentation that explains how to use the specific API. It provides details on available endpoints, request methods, request methods, required parameters, and response formats, often including code examples and testing tools. Good documentation helps developers to interact with the API efficiently.

The [GBIF API Reference documentation](https://techdocs.gbif.org/en/openapi/), for example, is split into serveral API sections. GBIF uses a RESTful API that can be accessed through structured URLs. A great feature of this documentation is that it’s built using [Swagger](https://swagger.io/), an interactive API framework. Swagger-based API pages let you test API queries before writing any code, making it easier to understand how they work. 

For example, the [GBIF Species API section](https://techdocs.gbif.org/en/openapi/v1/species#/Searching%20names/searchNames) allows to try the API directly in a web browser. Go to <a href="https://techdocs.gbif.org/en/openapi/v1/species#/Searching%20names/searchNames">https://techdocs.gbif.org/en/openapi/v1/species#/Searching%20names/searchNames</a> and use the general species search for "English Sparrow". Simply enter `English Sparrow` in the parameter field labeled `q` with the examplaination "The value for this parameter can be a simple word or a phrase. Wildcards are not supported" (Hint: The parameter field is located in the middle of the webpage). 
```

### Using the GBIF API in Python
We can search for a species' scientific name directly in Python. In the following code snippets, we load GBIF's search results for a given common name and extract the matching scientific names for `English Sparrow`.

We also specify a dataset to check this taxon. In this case, we use the base [GBIF Backbone Taxonomy](https://www.gbif.org/dataset/d7dddbf4-2cf0-4f39-9b2a-bb099caae36c).

In [18]:
search_name = "English Sparrow"
dataset_key = "d7dddbf4-2cf0-4f39-9b2a-bb099caae36c"
query_url = f'https://api.gbif.org/v1/species/search?q={search_name}&datasetKey={dataset_key}'

(content:references:admonition)=
```{admonition} Use your own common name!
:class: dropdown, attention
You can try this workflow with a different species! Simply replace `English Sparrow` with another common name to find its corresponding scientific name.
```

```{admonition} Syntax: f'{}' ?
:class: tip, dropdown
This is called an f-string, [a convenient python convention](https://realpython.com/python-f-strings/) to concat strings and variables.
```

Get the json-answer from the API (without login!).

In [19]:
import requests

json_text = None
response = requests.get(url=query_url)

The data can be found in `response.text` (in case the API answered).

In [20]:
import json
json_data = json.loads(response.text)

We can see that the correct answer, `Passer domesticus`, is hidden in the nested json structure:
(content:references:nub-id)=

In [22]:
print(json.dumps(json_data, indent=2)[0:550])

{
  "offset": 0,
  "limit": 20,
  "endOfRecords": true,
  "count": 1,
  "results": [
    {
      "key": 5231190,
      "nameKey": 8290258,
      "datasetKey": "d7dddbf4-2cf0-4f39-9b2a-bb099caae36c",
      "constituentKey": "7ddf754f-d193-4cc9-b351-99906754a03b",
      "nubKey": 5231190,
      "parentKey": 2492321,
      "parent": "Passer",
      "basionymKey": 8933000,
      "basionym": "Fringilla domestica Linnaeus, 1758",
      "kingdom": "Animalia",
      "phylum": "Chordata",
      "order": "Passeriformes",
      "family": "Passeridae",
   


We can access the most relevant first result and print this name directly. Below, we walk the json path from the `"results"`, which is a list, access the first entry by using `[0]` and the access its key of the name `scientificName`.

In [23]:
json_data["results"][0]["scientificName"]

'Passer domesticus (Linnaeus, 1758)'

We can also see the taxon id, a unique reference within the GBIF database that we will need in the next chapter to query occurrences:
(content:references:passer-id)=

In [24]:
json_data["results"][0]["taxonID"]

'gbif:5231190'

`````{admonition} Continue in the next section..
:class: seealso
*Passer domesticus* will be our example species for which we will retrieve observations [in the next section](202_data_retrieval).
`````

You can see a full list of services [here](https://monitor.ioer.de/monitor_api/services).

## References

```{bibliography}
:style: unsrt
:filter: docname in docnames
```

```{admonition} Recommended citation
:class: attention
IÖR Monitor (year). Dataset. URL. DOI.
```

 ```{admonition} Recommended citation
:class: attention




When downloading data from GBIF.org, a registered user is immediately redirected to a page that includes the following information:

GBIF.org (06 March 2025) GBIF Occurrence Download  https://doi.org/10.15468/dl.mvkpx9


GBIF (year). Dataset. URL. DOI.
```